In [3]:
import matplotlib 
import pandas as pd
import numpy as np
import math
import re
#import pymysql as db
import os
import sklearn
import datetime
from matplotlib import pyplot as plt
#from sshtunnel import SSHTunnelForwarder
from sklearn import model_selection, svm
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.metrics import classification_report, average_precision_score, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.pipeline import Pipeline

#BCI MNE Imports
import mne
from mne import Epochs, pick_types, find_events
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP


In [4]:

#Data Ingestion

raw_files = [read_raw_edf('C:\\648 wala\\VIVEK SHARMA\\EDUCATIONAL\\Brain Computer Interface\\DataSets\\Comptetion3\\k3b.gdf', preload=True, stim_channel='auto')]



Extracting edf Parameters from C:\648 wala\VIVEK SHARMA\EDUCATIONAL\Brain Computer Interface\DataSets\Comptetion3\k3b.gdf...
GDF file detected
Overlapping events detected. Use find_edf_events for the original events.
Setting channel info structure...
Interpolating stim channel. Events may jitter.
Created Raw.info structure...
Reading 0 ... 986779  =      0.000 ...  3947.116 secs...


<ipython-input-4-f4f567bb7e4c>:4: RuntimeWarning: Overlapping events detected. Use find_edf_events for the original events.
  raw_files = [read_raw_edf('C:\\648 wala\\VIVEK SHARMA\\EDUCATIONAL\\Brain Computer Interface\\DataSets\\Comptetion3\\k3b.gdf', preload=True, stim_channel='auto')]
<ipython-input-4-f4f567bb7e4c>:4: RuntimeWarning: Interpolating stim channel. Events may jitter.
  raw_files = [read_raw_edf('C:\\648 wala\\VIVEK SHARMA\\EDUCATIONAL\\Brain Computer Interface\\DataSets\\Comptetion3\\k3b.gdf', preload=True, stim_channel='auto')]


Ready.


In [5]:
raw_files[0]

<RawEDF  |  k3b.gdf, n_channels x n_times : 61 x 986780 (3947.1 sec), ~459.4 MB, data loaded>

[]

In [4]:
#Extracting Data From The raw_files
subject_list=[]
i=0
while True:
    if(i == 594):
        break
    a=raw_files[0+i:6+i]  
    subject_list.append(a)
    i=i+6



In [5]:
raw=raw_files[0]

In [6]:
raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')


Setting up band-pass filter from 7 - 30 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 7.5 Hz
Filter length of 413 samples (1.652 sec) selected


<RawEDF  |  k3b.gdf, n_channels x n_times : 61 x 986780 (3947.1 sec), ~459.4 MB, data loaded>

In [7]:
events = find_events(raw, shortest_event=0, stim_channel='STI 014')

_, pos, kind, chan, dur = raw_files[0].find_edf_events()

1080 events found
Events id: [ 768 1791 2323 2324 2325 2326 2337 2339 3346 3347 3348 3349 3360 3362]


In [8]:
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
event_id = dict(Lhand=2323,Rhand=2324, Feet=2325,Tongue=2326)
tmin, tmax = -1., 4.

In [9]:
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)

149 matching events found
0 projection items activated
Loading data for 149 events and 1251 original time points ...
0 bad epochs dropped


In [10]:
epochs_train = epochs.copy().crop(tmin=1., tmax=2.)

In [11]:
epochs_train

<Epochs  |  n_events : 149 (all good), tmin : 1.0 (s), tmax : 2.0 (s), baseline : None, ~17.2 MB, data loaded,
 'Feet': 38, 'Lhand': 36, 'Rhand': 37, 'Tongue': 38>

In [12]:
labels = epochs.events[:, -1]-2322
labels

array([3, 4, 3, 3, 3, 2, 4, 4, 1, 2, 2, 1, 2, 4, 1, 2, 3, 1, 1, 3, 1, 3, 1,
       1, 1, 2, 4, 3, 2, 4, 2, 2, 3, 4, 1, 2, 4, 3, 1, 4, 1, 4, 3, 1, 1, 2,
       1, 4, 4, 2, 2, 3, 4, 1, 4, 4, 3, 1, 3, 4, 3, 3, 3, 4, 2, 4, 4, 1, 1,
       3, 2, 4, 2, 1, 4, 2, 2, 4, 1, 2, 2, 4, 2, 3, 4, 4, 3, 3, 4, 2, 3, 2,
       1, 3, 4, 1, 2, 1, 1, 3, 4, 3, 3, 1, 3, 4, 2, 2, 2, 4, 4, 3, 3, 4, 1,
       3, 1, 3, 3, 1, 2, 2, 1, 1, 2, 2, 3, 4, 3, 1, 1, 2, 2, 1, 3, 3, 4, 2,
       3, 1, 1, 2, 4, 2, 4, 4, 4, 2, 3])

In [13]:
def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred))
    return accuracy_score(y_true, y_pred)


In [14]:
scores = []
epochs_data = epochs.get_data()
epochs_data_train = epochs_train.get_data()
cv = ShuffleSplit(5, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)

In [15]:
# Assemble a classifier
LDA = LinearDiscriminantAnalysis()
csp = CSP(n_components=5, reg=None, log=True, norm_trace=False)
clf = Pipeline([('CSP', csp), ('LDA', LDA)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1, scoring=make_scorer(classification_report_with_accuracy_score))


             precision    recall  f1-score   support

          1       0.86      1.00      0.92         6
          2       1.00      0.89      0.94         9
          3       0.80      0.67      0.73         6
          4       0.80      0.89      0.84         9

avg / total       0.87      0.87      0.87        30

             precision    recall  f1-score   support

          1       1.00      1.00      1.00         7
          2       1.00      1.00      1.00         6
          3       0.60      0.86      0.71         7
          4       0.86      0.60      0.71        10

avg / total       0.86      0.83      0.83        30

             precision    recall  f1-score   support

          1       1.00      0.75      0.86         4
          2       0.92      1.00      0.96        11
          3       0.78      0.88      0.82         8
          4       0.67      0.57      0.62         7

avg / total       0.83      0.83      0.83        30

             precision    recall  f1-

C:\Users\Vivek Sharma\AppData\Local\Programs\Python\Python36\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:455: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


In [16]:
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

Classification accuracy: 0.813333 / Chance level: 0.744966


In [30]:
LDA = LinearDiscriminantAnalysis()
csp = CSP(n_components=5, reg=None, log=True, norm_trace=False)
clf = Pipeline([('CSP', csp), ('LDA', LDA)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1, scoring=make_scorer(classification_report_with_accuracy_score))



             precision    recall  f1-score   support

          1       0.86      1.00      0.92         6
          2       1.00      0.89      0.94         9
          3       0.80      0.67      0.73         6
          4       0.80      0.89      0.84         9

avg / total       0.87      0.87      0.87        30

             precision    recall  f1-score   support

          1       1.00      1.00      1.00         7
          2       1.00      1.00      1.00         6
          3       0.60      0.86      0.71         7
          4       0.86      0.60      0.71        10

avg / total       0.86      0.83      0.83        30

             precision    recall  f1-score   support

          1       1.00      0.75      0.86         4
          2       0.92      1.00      0.96        11
          3       0.78      0.88      0.82         8
          4       0.67      0.57      0.62         7

avg / total       0.83      0.83      0.83        30

             precision    recall  f1-

C:\Users\Vivek Sharma\AppData\Local\Programs\Python\Python36\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:455: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


In [32]:
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
lda=(np.mean(scores))*100
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

Classification accuracy: 0.813333 / Chance level: 0.744966


In [33]:
lda

81.333333333333343

In [22]:
from sklearn.neighbors import KNeighborsClassifier

KNN=KNeighborsClassifier(n_neighbors=2)
clf = Pipeline([('CSP', csp), ('KNN', KNN)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1, scoring=make_scorer(classification_report_with_accuracy_score))






             precision    recall  f1-score   support

          1       0.67      1.00      0.80         6
          2       1.00      0.67      0.80         9
          3       0.67      0.67      0.67         6
          4       0.78      0.78      0.78         9

avg / total       0.80      0.77      0.77        30

             precision    recall  f1-score   support

          1       1.00      0.86      0.92         7
          2       0.86      1.00      0.92         6
          3       0.60      0.86      0.71         7
          4       0.86      0.60      0.71        10

avg / total       0.83      0.80      0.80        30

             precision    recall  f1-score   support

          1       0.67      1.00      0.80         4
          2       0.91      0.91      0.91        11
          3       0.70      0.88      0.78         8
          4       0.67      0.29      0.40         7

avg / total       0.76      0.77      0.74        30

             precision    recall  f1-

In [26]:
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
knn=np.mean(scores)*100
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))


Classification accuracy: 0.786667 / Chance level: 0.744966


In [27]:
print(knn)

78.6666666667


In [34]:
from sklearn.neural_network import MLPClassifier
MLP=MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
clf = Pipeline([('CSP', csp), ('MLP', MLP)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1, scoring=make_scorer(classification_report_with_accuracy_score))


             precision    recall  f1-score   support

          1       0.71      0.83      0.77         6
          2       0.89      0.89      0.89         9
          3       0.50      0.67      0.57         6
          4       0.67      0.44      0.53         9

avg / total       0.71      0.70      0.69        30

             precision    recall  f1-score   support

          1       1.00      0.86      0.92         7
          2       0.86      1.00      0.92         6
          3       0.43      0.86      0.57         7
          4       0.67      0.20      0.31        10

avg / total       0.73      0.67      0.64        30

             precision    recall  f1-score   support

          1       1.00      0.50      0.67         4
          2       0.50      0.09      0.15        11
          3       0.33      0.88      0.48         8
          4       0.60      0.43      0.50         7

avg / total       0.55      0.43      0.39        30

             precision    recall  f1-

In [35]:
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
mlp=np.mean(scores)*100
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

Classification accuracy: 0.613333 / Chance level: 0.744966


81.3333333333 78.6666666667 61.3333333333


In [37]:
from sklearn import svm
SVM=svm.SVC()
csp = CSP(n_components=5, reg=None, log=True, norm_trace=False)
clf = Pipeline([('CSP', csp), ('SVM', SVM)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1, scoring=make_scorer(classification_report_with_accuracy_score))



             precision    recall  f1-score   support

          1       0.62      0.83      0.71         6
          2       0.86      0.67      0.75         9
          3       1.00      0.67      0.80         6
          4       0.82      1.00      0.90         9

avg / total       0.83      0.80      0.80        30

             precision    recall  f1-score   support

          1       1.00      0.86      0.92         7
          2       0.86      1.00      0.92         6
          3       0.67      0.86      0.75         7
          4       0.88      0.70      0.78        10

avg / total       0.85      0.83      0.83        30

             precision    recall  f1-score   support

          1       1.00      1.00      1.00         4
          2       0.92      1.00      0.96        11
          3       0.70      0.88      0.78         8
          4       0.75      0.43      0.55         7

avg / total       0.83      0.83      0.82        30

             precision    recall  f1-

In [38]:
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
svm=np.mean(scores)*100
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

Classification accuracy: 0.793333 / Chance level: 0.744966


In [19]:
'''
subject_id=input("Please Enter the Subject ID From 0 - 99 For Classification:::  ")
subject_id_data=subject_list[int(subject_id)]

#Preprocessing Begin
tmin, tmax = -1., 4.
event_id = dict(hands=2, feet=3)

#Concatenation Of All the Files
raw = concatenate_raws(subject_id_data)

#Renaming of Data Extraction Channels
raw.rename_channels(lambda x: x.strip('.'))

#Applying Band Pass Filter From 7 -30 Hz Can Be Changed when needed
raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')

events = find_events(raw, shortest_event=0, stim_channel='STI 014')
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
# Testing will be done with a running classifier
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)
epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
labels = epochs.events[:, -1] - 2

#Cross Validation Function
def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred))
    return accuracy_score(y_true, y_pred)

scores = []
epochs_data = epochs.get_data()
epochs_data_train = epochs_train.get_data()
cv = ShuffleSplit(5, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)

# Assemble a classifier
LDA = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
clf = Pipeline([('CSP', csp), ('LDA', LDA)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1, scoring=make_scorer(classification_report_with_accuracy_score))


# Results
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

#Plot
csp.fit_transform(epochs_data, labels)

layout = read_layout('EEG1005')
csp.plot_patterns(epochs.info, layout=layout, ch_type='eeg', units='Patterns (AU)', size=1.5)
'''

'\nsubject_id=input("Please Enter the Subject ID From 0 - 99 For Classification:::  ")\nsubject_id_data=subject_list[int(subject_id)]\n\n#Preprocessing Begin\ntmin, tmax = -1., 4.\nevent_id = dict(hands=2, feet=3)\n\n#Concatenation Of All the Files\nraw = concatenate_raws(subject_id_data)\n\n#Renaming of Data Extraction Channels\nraw.rename_channels(lambda x: x.strip(\'.\'))\n\n#Applying Band Pass Filter From 7 -30 Hz Can Be Changed when needed\nraw.filter(7., 30., fir_design=\'firwin\', skip_by_annotation=\'edge\')\n\nevents = find_events(raw, shortest_event=0, stim_channel=\'STI 014\')\npicks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude=\'bads\')\n# Testing will be done with a running classifier\nepochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)\nepochs_train = epochs.copy().crop(tmin=1., tmax=2.)\nlabels = epochs.events[:, -1] - 2\n\n#Cross Validation Function\ndef classification_report_with_accuracy